In [1]:
import pandas as pd
import numpy as np

In [2]:
def remove_front_space(x):
    if x[0] == ' ':
        value = x[1:]
    else:
        value = x
    return value

# Import data

We have 7 data here: county_fips, state_fips, ihp_df, hma_df, cdbg_df, pa_federal_df, and disaster_df. I am going to clean each data and finally, I will merge all of this dataset.

In [3]:
# Import data
file_path = "data/"

county_fips = pd.read_excel(file_path + "fips_code.xlsx")
state_fips = pd.read_excel(file_path + "state_fips_code.xlsx")
ihp_df = pd.read_csv(file_path + "IHP.csv")
hma_df = pd.read_csv(file_path + "HMA.csv")
cdbg_df = pd.read_csv(file_path + 'HUD_Yearly_County_Funding_by_PDD_Years.csv')
pa_federal_df = pd.read_csv(file_path + "PA_PDD_County_Funding_Based_On_PDD_Name_Federal_Share_Obligated.csv")
disaster_df = pd.read_csv(file_path + "DisasterDeclarationsSummaries.csv")

# See all columns in dataframeZ
pd.set_option('display.max_columns', None)

# 1. Processing HMA(Hazard Mitigation Assistance) data

Let's start with the HMA data. This data has mitigation grant data. The main point here is that we are going to consider grants related to a disaster which got PDD(Presidential Decleared Disaster). 

In [4]:
# Explore the columns in hma dataset.
hma_df.dtypes

region                         int64
stateNumberCode                int64
state                         object
countyCode                   float64
county                        object
programArea                   object
projectIdentifier             object
projectType                   object
projectTitle                  object
projectCounties               object
numberOfProperties             int64
numberOfFinalProperties        int64
status                        object
subgrantee                    object
costSharePercentage          float64
projectAmount                float64
federalShareObligated        float64
programFy                      int64
dateInitiallyApproved         object
dateApproved                  object
dateClosed                    object
granteeTribalIndicator       float64
grantee                       object
disasterNumber               float64
benefitCostRatio             float64
netValueBenefits             float64
subgranteeTribalIndicator    float64
h

We are not interested in places located in outside of the main land such as Guam and Pueto Rico. For this reason, we are going to remove rows having grants related to places outside of the main land. Only 51 states' data will remain.

In [5]:
# Change the name of column from 'fips_code' to 'FIPS'
state_fips = state_fips.rename({'fips_code': 'FIPS'}, axis = 1)

# Get only 51 states in the main land
state_fips_code = state_fips["FIPS"].unique()
hma_df = hma_df[hma_df["stateNumberCode"].isin(state_fips_code)]

Now, we have only 51 states' data!

In [6]:
len(hma_df["stateNumberCode"].unique())

51

#### projectIdentifier
Disaster-based projects use the convention of DR-disaster number-project number-suffix (e.g., DR-1761-0001-M) while non-disaster projects use the unique project number (e.g., FMA-PJ-10-WA-2017-006)

In [7]:
no_disasterNumber = hma_df[hma_df["disasterNumber"].isna()]
no_disasterNumber["programArea"].unique()

array(['FMA', 'PDM', 'LPDM', 'RFC', 'SRL'], dtype=object)

In [8]:
yes_disasterNumber = hma_df[hma_df["disasterNumber"].notna()]
yes_disasterNumber["programArea"].unique()

array(['HMGP', 'PDM'], dtype=object)

Above result tells us that only disaster projects only provides HMGP and PDM grants. We are going to use data related to disaster project(disaster with PDD: Presidential Declared Disaster)

In [9]:
hma_df = hma_df[hma_df["disasterNumber"].notna()]

#### County

Here is the explanation about county in the data dictionary provided by official FEMA website:
Full name of the primary county (e.g., Broward) associated with the project. Note, projects can be associated with more than one county (see the Project Counties field) and are often reported as Statewide (e.g., 000 represents Statewide)

This means even though many counties are related to a project, only main one county is stated in county. For this reason, if there is more than one counties are listed in the "projectCounties" column, I am going to split the row into several rows and divide the amount of project money by the number of counties.

For example, the below shows an example. 25 counties are related to a project.

In [10]:
print(hma_df.loc[3]["projectCounties"])
print(len(hma_df.loc[3]["projectCounties"]))

ONTARIO; ONONDAGA; RENSSELAER; SARATOGA; SCHENECTADY; SCHOHARIE; SCHUYLER; SENECA; TOMPKINS; WARREN; WAYNE; YATES; ALBANY; CAYUGA; CHEMUNG; ESSEX; FULTON; GREENE; HAMILTON; HERKIMER; LEWIS; MADISON; MONTGOMERY; ONEIDA; OSWEGO
225


The code below calculate the number of counties related to each grant project.

In [11]:
# Calculate the number of counties related to a project
hma_df["county_num"] = hma_df["projectCounties"].apply(lambda x: str(x).count(';') + 1)

In [12]:
print(hma_df[hma_df["county_num"] == 1].shape)
print(hma_df[hma_df["county_num"] > 1].shape)

(22008, 31)
(478, 31)


There are 22008 rows having only one county, and 478 rows having more than one county. We are going to split the latter one.

To process data, let's divide data into two:
1. county_one: rows having only one county in "projectCounties" column
2. county_many: rows having many counties, more than one, in "projectCounties" column

In [13]:
county_one = hma_df[hma_df["county_num"] == 1]
county_many = hma_df[hma_df["county_num"] != 1]

So now we have two dataset for hma_df. And we are going to split "county_many" dataframe into two:
- county_many_statewide: The grant is for all counties in a state.
- county_many_non_statewide: The grant is NOT for all counties in a state.

In summurize, there are 3 datasets for hma data.
1. county_one
2. county_many - county_many_statewide
               - county_many_non_statewide

In [14]:
county_many_statewide = county_many[county_many["grantee"] == "Statewide"]
county_many_non_statewide = county_many[county_many["grantee"] != "Statewide"]

In [15]:
# Remove the blank space behind the letter.
county_many_non_statewide["projectCounties"] = county_many_non_statewide["projectCounties"].apply(remove_front_space)

<ipython-input-15-aa8e4f50bfdd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_many_non_statewide["projectCounties"] = county_many_non_statewide["projectCounties"].apply(remove_front_space)


For 'county_many_statewide' dataframe, we will assign 'Statewide' to county_name, and '0' to 'countyCode'

In [16]:
county_many_statewide["county"] = "Statewide"
county_many_statewide["countyCode"] = 0

<ipython-input-16-90ef86297946>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_many_statewide["county"] = "Statewide"
<ipython-input-16-90ef86297946>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_many_statewide["countyCode"] = 0


## county_many_non_statewide

In [18]:
# Having only important columns
county_many_non_statewide = county_many_non_statewide[["stateNumberCode", "state", "countyCode", "county", "programArea", "projectCounties", "federalShareObligated", "programFy", "disasterNumber", "county_num"]]

Let's divde the rows into many rows so that one row has only one county. For example, if there is row of which projectCounties includes A, B, C counties. Then we are goint to split this row into 3 rows. Also, we are going to divide the amount by the number of related counties and assign it to the projectAmount.

In [19]:
county_many_non_statewide = county_many_non_statewide.set_index(['stateNumberCode', 'state', 'countyCode', 'county', 'programArea', 'federalShareObligated', 'disasterNumber', 'county_num', 'programFy']).apply(lambda x: x.str.split(';').explode()).reset_index()

In [20]:
county_many_non_statewide["county"] = county_many_non_statewide["county"].apply(remove_front_space)
county_many_non_statewide["projectCounties"] = county_many_non_statewide["projectCounties"].apply(remove_front_space)

In [21]:
# Convert the projectCounties into capitalized letters
county_many_non_statewide['county'] = county_many_non_statewide["projectCounties"].str.capitalize()

In [23]:
county_many_non_statewide.loc[county_many_non_statewide["county"] != "Statewide", 'federalShareObligated'] = county_many_non_statewide["federalShareObligated"] / county_many_non_statewide["county_num"]

In [24]:
county_many_non_statewide.head()

,stateNumberCode,state,countyCode,county,programArea,federalShareObligated,disasterNumber,county_num,programFy,projectCounties
0,46,South Dakota,47.0,Oglala lakota,HMGP,16500.0,4237.0,3,2015,OGLALA LAKOTA
1,46,South Dakota,47.0,Bennett,HMGP,16500.0,4237.0,3,2015,BENNETT
2,46,South Dakota,47.0,Fall river,HMGP,16500.0,4237.0,3,2015,FALL RIVER
3,37,North Carolina,173.0,Jackson,HMGP,46594.5,4103.0,2,2013,JACKSON
4,37,North Carolina,173.0,Swain,HMGP,46594.5,4103.0,2,2013,SWAIN


There are some rows of which the value of "county" is not actually county. Some rows have city name for this value. In this case, I am going to manually change their county code and county name.

In [25]:
county_many_non_statewide.loc[county_many_non_statewide.county == 'Oglala lakota', "countyCode"] = 102.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Fall river', "countyCode"] = 47.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Bennett', "countyCode"] = 7.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Jackson', "countyCode"] = 102.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Swain', "countyCode"] = 173.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Graham', "countyCode"] = 75.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Cherokee', "countyCode"] = 39.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Haywood', "countyCode"] = 87.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Hill', "countyCode"] = 41.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Chouteau', "countyCode"] = 15.0
county_many_non_statewide.loc[county_many_non_statewide.county == 'Monona', "countyCode"] = 43.0

In [26]:
county_many_non_statewide.head()

,stateNumberCode,state,countyCode,county,programArea,federalShareObligated,disasterNumber,county_num,programFy,projectCounties
0,46,South Dakota,102.0,Oglala lakota,HMGP,16500.0,4237.0,3,2015,OGLALA LAKOTA
1,46,South Dakota,7.0,Bennett,HMGP,16500.0,4237.0,3,2015,BENNETT
2,46,South Dakota,47.0,Fall river,HMGP,16500.0,4237.0,3,2015,FALL RIVER
3,37,North Carolina,102.0,Jackson,HMGP,46594.5,4103.0,2,2013,JACKSON
4,37,North Carolina,173.0,Swain,HMGP,46594.5,4103.0,2,2013,SWAIN


Now 'county_many_non_statewide' dataframe is ready to be merged!

In [27]:
# Leaving only important columns
county_many_statewide = county_many_statewide[["stateNumberCode", "state", "countyCode", "county", "programArea", "federalShareObligated",  "disasterNumber", "county_num", "programFy", "projectCounties"]]
county_one = county_one[["stateNumberCode", "state", "countyCode", "county", "programArea", "federalShareObligated",  "disasterNumber", "county_num", "programFy", "projectCounties"]]

## county_one

Let's divide 'county_one' dataframe into two:
- county_one_no_countyCode: The row has only one county but it does NOT have a countycode
- county_one_countyCode: The row has only one county but it DOES have a countycode

In [28]:
county_one_no_countyCode = county_one[county_one["countyCode"].isna()]
county_one_countyCode = county_one[county_one["countyCode"].notna()]

After that, we are going to divide the 'county_one_no_countyCode' into two dataframe:
- county_one_no_countyCode_statewide: county name is statewide
- county_one_no_countyCode_no_statewide: county name is not statewide

In [37]:
county_one_no_countyCode_statewide = county_one_no_countyCode[county_one_no_countyCode["county"] == "Statewide"]
county_one_no_countyCode_no_statewide = county_one_no_countyCode[county_one_no_countyCode["county"] != "Statewide"]

## county_one_no_countyCode_statewide

In [38]:
# If the county is "statewide", we are assigninig 0 to countyCode.
county_one_no_countyCode_statewide["countyCode"] = 0.0

In [39]:
county_one_no_countyCode_no_statewide["county"] = county_one_no_countyCode_no_statewide["county"].str.title()

In [40]:
county_one_no_countyCode_no_statewide.head()

,stateNumberCode,state,countyCode,county,programArea,federalShareObligated,disasterNumber,county_num,programFy,projectCounties
2864,41,Oregon,NaN,NaN,HMGP,369934.0,1099.0,1,1996,MARION
3090,50,Vermont,NaN,NaN,HMGP,11813.0,1184.0,1,1997,CALADONIA
3101,22,Louisiana,NaN,NaN,PDM,32250.0,9031.0,1,2002,WEST CARROLL
3106,16,Idaho,NaN,NaN,HMGP,292074.0,1102.0,1,1996,AHOSHONE
3209,48,Texas,NaN,NaN,HMGP,5000.0,828.0,1,1989,HENDERSONN


In [41]:
# Change the column names.
county_fips = county_fips.rename({'state_code': 'stateNumberCode', 'county_code': 'countyCode', 'name': 'county'}, axis = 1)

In [42]:
# Add countyCode to 'county_one_no_countyCode_no_statewide'
add_countyCode = pd.merge(county_one_no_countyCode_no_statewide, county_fips, how = "left", on = ["stateNumberCode", "county"])

I am going to divide "add_countyCode" dataset into "add_countyCode_done", which does not need the process adding countyCode, andn "add_countyCode_need_cc", which requires the process adding county codes.

In [43]:
add_countyCode_done = add_countyCode[add_countyCode["countyCode_y"].notna()]
add_countyCode_need_cc = add_countyCode[add_countyCode["countyCode_y"].isna()]

In [44]:
add_countyCode_done = add_countyCode_done[["stateNumberCode", "state", "countyCode_y", "county", "programArea", "federalShareObligated",  "disasterNumber", "county_num", "programFy", "projectCounties"]]

In [45]:
add_countyCode_done = add_countyCode_done.rename({'countyCode_y': 'countyCode'}, axis = 1)

In [ ]:
#add_countyCode_need_cc.to_csv("~/hma_need_countyCode.csv")

Since there are some rows of which county name is not correct. (Some rows have city or town name for county name.) In this case, I manually added actual county names.

In [46]:
# Import manually processed data
add_countyCode_need_cc_after = pd.read_csv(file_path + "hma_need_countyCode.csv")
add_countyCode_need_cc_after = add_countyCode_need_cc_after.drop(columns = ["Unnamed: 0"])
add_countyCode_need_cc_after = add_countyCode_need_cc_after[["stateNumberCode", "state", "countyCode_x", "county", "programArea", "federalShareObligated",  "disasterNumber", "county_num", "programFy", "projectCounties"]]
add_countyCode_need_cc_after = add_countyCode_need_cc_after.rename({'countyCode_x' : 'countyCode'}, axis = 1)

In [47]:
add_countyCode_need_cc_after.head()

,stateNumberCode,state,countyCode,county,programArea,federalShareObligated,disasterNumber,county_num,programFy,projectCounties
0,50,Vermont,5.0,Caledonia,HMGP,11813.0,1184.0,1.0,1997,CALADONIA
1,16,Idaho,79.0,Shoshone,HMGP,292074.0,1102.0,1.0,1996,AHOSHONE
2,48,Texas,77.0,Henderson,HMGP,5000.0,828.0,1.0,1989,HENDERSONN
3,21,Kentucky,191.0,Pendleton,HMGP,323023.0,1163.0,1.0,1997,PENDELTON
4,38,North Dakota,15.0,Burleigh,HMGP,77615.0,1050.0,1.0,1995,BISMARCK


Okay, now we are going to merge all splited data so that one dataframe has all hma data. There are 6 dataframes we need to merge: 'county_one_no_countyCode_statewide', 'add_countyCode_done', 'add_countyCode_need_cc_after', 'county_one_countyCode', 'county_many_statewide', and 'county_many_non_statewide'

In [50]:
# Drop unnecessary columns
drop_cols = ["county_num", "projectCounties"]
add_countyCode_done = add_countyCode_done.drop(columns = drop_cols)
add_countyCode_need_cc_after = add_countyCode_need_cc_after.drop(columns = drop_cols)
county_one_countyCode = county_one_countyCode.drop(columns = drop_cols)
county_many_statewide = county_many_statewide.drop(columns = drop_cols)
county_many_non_statewide = county_many_non_statewide.drop(columns = drop_cols)

In [51]:
# Merge all dataframe
final_hma_df = pd.concat([county_one_no_countyCode_statewide, add_countyCode_done, add_countyCode_need_cc_after, county_one_countyCode, county_many_statewide, county_many_non_statewide], ignore_index = True)

In [52]:
# Drop 'county' column
final_hma_df_temp = final_hma_df.drop(columns = ['county'])

In [53]:
final_hma_df_temp_hmgp = final_hma_df_temp[final_hma_df_temp["programArea"] == "HMGP"]
final_hma_df_temp_pdm = final_hma_df_temp[final_hma_df_temp["programArea"] == "PDM"]

In order to group by same FIPS code, disasterNumber and programFy, let's divide the data into two by programArea.
- The dataset having only hmgp grants
- The dataset having only pdm grants.

In [54]:
final_hma_df_temp_hmgp = final_hma_df_temp_hmgp.groupby(["stateNumberCode", "state", "countyCode", "programArea", "disasterNumber", "programFy"])["federalShareObligated"].sum().to_frame(name = "hmgp").reset_index()
final_hma_df_temp_pdm = final_hma_df_temp_pdm.groupby(["stateNumberCode", "state", "countyCode", "programArea", "disasterNumber", "programFy"])["federalShareObligated"].sum().to_frame(name = "pdm").reset_index()

final_hma_df_temp_hmgp = final_hma_df_temp_hmgp.drop(columns = ["programArea"])
final_hma_df_temp_pdm = final_hma_df_temp_pdm.drop(columns = ["programArea"])

In [55]:
# Merge two dataframe.
hma_df_merge = pd.merge(final_hma_df_temp_hmgp, final_hma_df_temp_pdm, on = ["stateNumberCode", "state", "countyCode", "disasterNumber", 'programFy'], how = 'outer')

In hmgp and pdm columns, let's fill NA with 0.

In [56]:
hma_df_merge["hmgp"] = hma_df_merge["hmgp"].fillna(0)
hma_df_merge["pdm"] = hma_df_merge["pdm"].fillna(0)

In [59]:
hma_df_merge.head()

,stateNumberCode,state,countyCode,disasterNumber,programFy,hmgp,pdm
0,1,Alabama,0.0,848.0,1990,15794.0,0.0
1,1,Alabama,0.0,856.0,1990,22597.0,0.0
2,1,Alabama,0.0,861.0,1990,84583.0,0.0
3,1,Alabama,0.0,1013.0,1994,267669.0,0.0
4,1,Alabama,0.0,1019.0,1994,200738.0,0.0


# Processing IHP data
First of all, I am going to remove all rows of which value is 0.

In [60]:
# Remove rows of which the amount of ihp is 0
ihp_df = ihp_df[ihp_df["sum.ihpAmount."] != 0]

In [61]:
# Getting disasterNumber 
disasterNumber_list = disaster_df["disasterNumber"].unique()

In [62]:
# change the column name
ihp_df = ihp_df.rename({'sum.ihpAmount.CPIAdjusted': 'ihp_cpi', 'DisasterYear': 'programFy', 'damagedStateAbbreviation': 'state', 'sum.ihpAmount.': 'ihp'}, axis = 1)

In [63]:
# Get only important columns
ihp_df = ihp_df[["incidentType", "programFy", "disasterNumber", "county", "state", "ihp", "FIPS", 'ihp_cpi']]

In [64]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [65]:
ihp_df = ihp_df[ihp_df["state"].isin(states)]

In [66]:
ihp_df.head()

,incidentType,programFy,disasterNumber,county,state,ihp,FIPS,ihp_cpi
0,Severe Storm(s),2002,1439,Aransas (County),TX,1544582.27,48007.0,2.195384e+06
1,Severe Storm(s),2002,1439,Brazoria (County),TX,277904.75,48039.0,3.949985e+05
2,Severe Storm(s),2002,1439,Cameron (County),TX,1320679.44,48061.0,1.877141e+06
3,Severe Storm(s),2002,1439,Fort Bend (County),TX,170743.37,48157.0,2.426852e+05
4,Severe Storm(s),2002,1439,Hardin (County),TX,547459.52,48199.0,7.781288e+05


I am going to divide ihp_df into two dataframe to clean and process data:
- need_county_ihp_df: it does NOT have fips code
- done_county_ihp_df: it DOES have fips code.

In [67]:
need_county_ihp_df = ihp_df[ihp_df["FIPS"].isna()]
done_county_ihp_df = ihp_df[ihp_df["FIPS"].notna()]

## done_county_ihp_df
This dataframe does NOT need a process to add countyCode.

In [68]:
done_county_ihp_df.dtypes

incidentType       object
programFy           int64
disasterNumber      int64
county             object
state              object
ihp               float64
FIPS              float64
ihp_cpi           float64
dtype: object

In [69]:
def correct_fips(col):
    if col < 10000:
        fip = "0" + str(int(col))
    else:
        fip = str(int(col))
    return fip

In [70]:
done_county_ihp_df.head()

,incidentType,programFy,disasterNumber,county,state,ihp,FIPS,ihp_cpi
0,Severe Storm(s),2002,1439,Aransas (County),TX,1544582.27,48007.0,2.195384e+06
1,Severe Storm(s),2002,1439,Brazoria (County),TX,277904.75,48039.0,3.949985e+05
2,Severe Storm(s),2002,1439,Cameron (County),TX,1320679.44,48061.0,1.877141e+06
3,Severe Storm(s),2002,1439,Fort Bend (County),TX,170743.37,48157.0,2.426852e+05
4,Severe Storm(s),2002,1439,Hardin (County),TX,547459.52,48199.0,7.781288e+05


In [71]:
# Change the format of "FIPS". ex) 1141.0 > 01141
done_county_ihp_df["FIPS"] = done_county_ihp_df["FIPS"].apply(correct_fips)

<ipython-input-71-823ae7e9b1ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  done_county_ihp_df["FIPS"] = done_county_ihp_df["FIPS"].apply(correct_fips)


## need_county_ihp_df
This dataframe DOES need a process to add county code.

In [72]:
# Remove parenthesis ex) (county)
need_county_ihp_df["county"] = need_county_ihp_df["county"].str.replace(r"\(.*\)","")

<ipython-input-72-059277ee11a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_county_ihp_df["county"] = need_county_ihp_df["county"].str.replace(r"\(.*\)","")


In [73]:
state_fips = state_fips.rename({'state_ab': 'state', 'FIPS': 'stateNumberCode'}, axis = 1)

In [74]:
state_fips_temp = state_fips[["state", "stateNumberCode"]]

In [75]:
need_county_ihp_df_stateCode = pd.merge(need_county_ihp_df, state_fips_temp, on = ['state'], how = 'left' )

In [76]:
need_county_ihp_df_stateCode.head()

,incidentType,programFy,disasterNumber,county,state,ihp,FIPS,ihp_cpi,stateNumberCode
0,Severe Storm(s),2002,1445,Chignik Lagoon,AK,2400.00,NaN,3.411228e+03,2
1,Severe Storm(s),2003,1458,Norton,VA,40413.14,NaN,5.615067e+04,51
2,Severe Storm(s),2003,1458,Roanoke,VA,215919.05,NaN,3.000014e+05,51
3,Severe Storm(s),2003,1458,Salem,VA,827593.62,NaN,1.149872e+06,51
4,Severe Storm(s),2003,1461,Anchorage,AK,140160.01,NaN,1.947406e+05,2


In [77]:
def remove_back_space(x):
    if x[-1] == ' ':
        value = x[:-1]
    else:
        value = x
    return value

In [78]:
need_county_ihp_df_stateCode["county"] = need_county_ihp_df_stateCode["county"].apply(remove_back_space)

In [79]:
need_county_ihp_df_stateCode.head()

,incidentType,programFy,disasterNumber,county,state,ihp,FIPS,ihp_cpi,stateNumberCode
0,Severe Storm(s),2002,1445,Chignik Lagoon,AK,2400.00,NaN,3.411228e+03,2
1,Severe Storm(s),2003,1458,Norton,VA,40413.14,NaN,5.615067e+04,51
2,Severe Storm(s),2003,1458,Roanoke,VA,215919.05,NaN,3.000014e+05,51
3,Severe Storm(s),2003,1458,Salem,VA,827593.62,NaN,1.149872e+06,51
4,Severe Storm(s),2003,1461,Anchorage,AK,140160.01,NaN,1.947406e+05,2


In [80]:
need_county_ihp_df_stateCode_countyCode = pd.merge(need_county_ihp_df_stateCode, county_fips, on = ['stateNumberCode', 'county'], how = 'left' )

In [81]:
need_countyCode_ihp_df = need_county_ihp_df_stateCode_countyCode[need_county_ihp_df_stateCode_countyCode["countyCode"].isna()]

In [82]:
need_countyCode_ihp_df_have_countyCode = need_county_ihp_df_stateCode_countyCode[need_county_ihp_df_stateCode_countyCode["countyCode"].notna()]

In [83]:
#need_countyCode_ihp_df.to_csv("/Users/soyeon/Desktop/need_countyCode_ihp_df.csv")

Same as before, there are some rows of which the value of county name is not actually county name. In this case, I manually changed their city name to county name and added county code.

In [84]:
ihp_df1 = pd.read_csv(file_path + "need_countyCode_ihp_df.csv")
ihp_df1 = ihp_df1.drop(columns = ["Unnamed: 0"])

In [85]:
def county_check(col):
    if col > 99:
        county_cd = str(col)
    elif col > 9:
        county_cd = "0" + str(col)
    else:
        county_cd = "00" + str(col)
    
    return county_cd

def state_check(col):
    if col > 9:
        state_cd = str(col)
    else:
        state_cd = "0" + str(col)
        
    return state_cd

In [86]:
ihp_df1["state_FIPS"] = ihp_df1["stateNumberCode"].apply(state_check)
ihp_df1["county_FIPS"] = ihp_df1["countyCode"].apply(county_check)

In [87]:
ihp_df1

,incidentType,programFy,disasterNumber,county,state,ihp,FIPS,ihp_cpi,stateNumberCode,countyCode,state_FIPS,county_FIPS
0,Severe Storm(s),2002,1445,Lake and Peninsula,AK,2400.00,NaN,3.411228e+03,2,164,02,164
1,Severe Storm(s),2003,1458,Norton,VA,40413.14,NaN,5.615067e+04,51,720,51,720
2,Severe Storm(s),2003,1458,Salem,VA,827593.62,NaN,1.149872e+06,51,775,51,775
3,Severe Storm(s),2003,1461,Anchorage,AK,140160.01,NaN,1.947406e+05,2,20,02,020
4,Hurricane,2003,1491,Alexandria,VA,224526.47,NaN,3.119607e+05,51,510,51,510
...,...,...,...,...,...,...,...,...,...,...,...,...
110,Biological,2020,4529,DeBaca,NM,8953.45,NaN,8.843056e+03,35,11,35,011
111,Biological,2020,4529,Dona Ana,NM,486839.01,NaN,4.808364e+05,35,13,35,013
112,Biological,2020,4533,Anchorage,AK,175692.63,NaN,1.735264e+05,2,20,02,020
113,Hurricane,2020,4559,La Salle,LA,522586.05,NaN,5.161427e+05,22,59,22,059


In [88]:
ihp_df1["FIPS"] = ihp_df1["state_FIPS"] + ihp_df1["county_FIPS"]

In [89]:
need_countyCode_ihp_df_have_countyCode

,incidentType,programFy,disasterNumber,county,state,ihp,FIPS,ihp_cpi,stateNumberCode,countyCode
35,Hurricane,2003,1493,District of Columbia,DC,953918.36,NaN,1325389.636,11,0.0
36,Hurricane,2003,1493,District of Columbia,DC,953918.36,NaN,1325389.636,11,1.0
153,Biological,2020,4502,District of Columbia,DC,2560945.40,NaN,2529369.414,11,0.0
154,Biological,2020,4502,District of Columbia,DC,2560945.40,NaN,2529369.414,11,1.0


In [90]:
need_countyCode_ihp_df_have_countyCode["countyCode"] = need_countyCode_ihp_df_have_countyCode["countyCode"].astype(int)

<ipython-input-90-e88e4402c3d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_countyCode_ihp_df_have_countyCode["countyCode"] = need_countyCode_ihp_df_have_countyCode["countyCode"].astype(int)


In [91]:
need_countyCode_ihp_df_have_countyCode

,incidentType,programFy,disasterNumber,county,state,ihp,FIPS,ihp_cpi,stateNumberCode,countyCode
35,Hurricane,2003,1493,District of Columbia,DC,953918.36,NaN,1325389.636,11,0
36,Hurricane,2003,1493,District of Columbia,DC,953918.36,NaN,1325389.636,11,1
153,Biological,2020,4502,District of Columbia,DC,2560945.40,NaN,2529369.414,11,0
154,Biological,2020,4502,District of Columbia,DC,2560945.40,NaN,2529369.414,11,1


In [92]:
need_countyCode_ihp_df_have_countyCode["state_FIPS"] = need_countyCode_ihp_df_have_countyCode["stateNumberCode"].apply(state_check)
need_countyCode_ihp_df_have_countyCode["county_FIPS"] = need_countyCode_ihp_df_have_countyCode["countyCode"].apply(county_check)

<ipython-input-92-78a0d40b792f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_countyCode_ihp_df_have_countyCode["state_FIPS"] = need_countyCode_ihp_df_have_countyCode["stateNumberCode"].apply(state_check)
<ipython-input-92-78a0d40b792f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_countyCode_ihp_df_have_countyCode["county_FIPS"] = need_countyCode_ihp_df_have_countyCode["countyCode"].apply(county_check)


In [93]:
need_countyCode_ihp_df_have_countyCode["FIPS"] = need_countyCode_ihp_df_have_countyCode["state_FIPS"] + need_countyCode_ihp_df_have_countyCode["county_FIPS"]

<ipython-input-93-024e185f33a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_countyCode_ihp_df_have_countyCode["FIPS"] = need_countyCode_ihp_df_have_countyCode["state_FIPS"] + need_countyCode_ihp_df_have_countyCode["county_FIPS"]


In [94]:
need_countyCode_ihp_df_have_countyCode

,incidentType,programFy,disasterNumber,county,state,ihp,FIPS,ihp_cpi,stateNumberCode,countyCode,state_FIPS,county_FIPS
35,Hurricane,2003,1493,District of Columbia,DC,953918.36,11000,1325389.636,11,0,11,000
36,Hurricane,2003,1493,District of Columbia,DC,953918.36,11001,1325389.636,11,1,11,001
153,Biological,2020,4502,District of Columbia,DC,2560945.40,11000,2529369.414,11,0,11,000
154,Biological,2020,4502,District of Columbia,DC,2560945.40,11001,2529369.414,11,1,11,001


In [95]:
done_county_ihp_df.dtypes

incidentType       object
programFy           int64
disasterNumber      int64
county             object
state              object
ihp               float64
FIPS               object
ihp_cpi           float64
dtype: object

In [96]:
done_county_ihp_df["stateNumberCode"] = done_county_ihp_df["FIPS"].str[:2].astype(int)
done_county_ihp_df["countyCode"] = done_county_ihp_df["FIPS"].str[-3:].astype(int)

<ipython-input-96-e45c46eae214>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  done_county_ihp_df["stateNumberCode"] = done_county_ihp_df["FIPS"].str[:2].astype(int)
<ipython-input-96-e45c46eae214>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  done_county_ihp_df["countyCode"] = done_county_ihp_df["FIPS"].str[-3:].astype(int)


In [97]:
done_county_ihp_df

,incidentType,programFy,disasterNumber,county,state,ihp,FIPS,ihp_cpi,stateNumberCode,countyCode
0,Severe Storm(s),2002,1439,Aransas (County),TX,1544582.27,48007,2.195384e+06,48,7
1,Severe Storm(s),2002,1439,Brazoria (County),TX,277904.75,48039,3.949985e+05,48,39
2,Severe Storm(s),2002,1439,Cameron (County),TX,1320679.44,48061,1.877141e+06,48,61
3,Severe Storm(s),2002,1439,Fort Bend (County),TX,170743.37,48157,2.426852e+05,48,157
4,Severe Storm(s),2002,1439,Hardin (County),TX,547459.52,48199,7.781288e+05,48,199
...,...,...,...,...,...,...,...,...,...,...
9474,Flood,2021,4606,East Baton Rouge (Parish),LA,3660437.48,22033,NaN,22,33
9475,Flood,2021,4606,Iberville (Parish),LA,743109.49,22047,NaN,22,47
9476,Flood,2021,4606,Lafayette (Parish),LA,485006.02,22055,NaN,22,55
9477,Severe Storm(s),2021,4607,Washtenaw (County),MI,649178.73,26161,NaN,26,161


In [98]:
done_county_ihp_df.dtypes

incidentType        object
programFy            int64
disasterNumber       int64
county              object
state               object
ihp                float64
FIPS                object
ihp_cpi            float64
stateNumberCode      int64
countyCode           int64
dtype: object

In [99]:
cols = ["stateNumberCode","countyCode", "FIPS", "programFy", "disasterNumber", "ihp", "ihp_cpi"]
done_county_ihp_df = done_county_ihp_df[cols]

In [100]:
done_county_ihp_df

,stateNumberCode,countyCode,FIPS,programFy,disasterNumber,ihp,ihp_cpi
0,48,7,48007,2002,1439,1544582.27,2.195384e+06
1,48,39,48039,2002,1439,277904.75,3.949985e+05
2,48,61,48061,2002,1439,1320679.44,1.877141e+06
3,48,157,48157,2002,1439,170743.37,2.426852e+05
4,48,199,48199,2002,1439,547459.52,7.781288e+05
...,...,...,...,...,...,...,...
9474,22,33,22033,2021,4606,3660437.48,NaN
9475,22,47,22047,2021,4606,743109.49,NaN
9476,22,55,22055,2021,4606,485006.02,NaN
9477,26,161,26161,2021,4607,649178.73,NaN


In [101]:
need_countyCode_ihp_df_have_countyCode = need_countyCode_ihp_df_have_countyCode[cols]

In [102]:
need_countyCode_ihp_df_have_countyCode

,stateNumberCode,countyCode,FIPS,programFy,disasterNumber,ihp,ihp_cpi
35,11,0,11000,2003,1493,953918.36,1325389.636
36,11,1,11001,2003,1493,953918.36,1325389.636
153,11,0,11000,2020,4502,2560945.40,2529369.414
154,11,1,11001,2020,4502,2560945.40,2529369.414


In [103]:
ihp_df1 = ihp_df1[cols]

In [104]:
ihp_df1

,stateNumberCode,countyCode,FIPS,programFy,disasterNumber,ihp,ihp_cpi
0,2,164,02164,2002,1445,2400.00,3.411228e+03
1,51,720,51720,2003,1458,40413.14,5.615067e+04
2,51,775,51775,2003,1458,827593.62,1.149872e+06
3,2,20,02020,2003,1461,140160.01,1.947406e+05
4,51,510,51510,2003,1491,224526.47,3.119607e+05
...,...,...,...,...,...,...,...
110,35,11,35011,2020,4529,8953.45,8.843056e+03
111,35,13,35013,2020,4529,486839.01,4.808364e+05
112,2,20,02020,2020,4533,175692.63,1.735264e+05
113,22,59,22059,2020,4559,522586.05,5.161427e+05


In [105]:
final_ihp_df = pd.concat([ihp_df1, need_countyCode_ihp_df_have_countyCode, done_county_ihp_df], ignore_index = True)

In [106]:
final_ihp_df_ihp = final_ihp_df[["stateNumberCode", "countyCode", "FIPS", "programFy", "disasterNumber", "ihp"]]

In [107]:
final_ihp_df_ihp_cpi = final_ihp_df[["stateNumberCode", "countyCode", "FIPS", "programFy", "disasterNumber", "ihp_cpi"]]

In [108]:
final_ihp_df_ihp_temp = final_ihp_df_ihp.groupby(["stateNumberCode", "countyCode", "FIPS", "programFy", "disasterNumber"])["ihp"].sum().to_frame(name = "ihp").reset_index()

In [109]:
final_ihp_df_ihp_cpi_temp = final_ihp_df_ihp_cpi.groupby(["stateNumberCode", "countyCode", "FIPS", "programFy", "disasterNumber"])["ihp_cpi"].sum().to_frame(name = "ihp_cpi").reset_index()

In [110]:
final_ihp_df = pd.merge(final_ihp_df_ihp_temp, final_ihp_df_ihp_cpi_temp, on = ["stateNumberCode", "countyCode", "FIPS", "programFy", "disasterNumber"], how = 'inner')

In [111]:
final_ihp_df.head()

,stateNumberCode,countyCode,FIPS,programFy,disasterNumber,ihp,ihp_cpi
0,1,1,01001,2004,1549,1664427.61,2.252496e+06
1,1,1,01001,2009,1842,494688.11,5.894181e+05
2,1,1,01001,2011,1971,909158.89,1.033371e+06
3,1,1,01001,2020,4503,215766.63,2.131063e+05
4,1,3,01003,2003,1466,1993273.62,2.769487e+06


# Processing PA data

In [112]:
# Remove rows of which grant is 0
pa_federal_df = pa_federal_df.loc[:, (pa_federal_df != 0).any(axis=0)]

In [113]:
pa_federal_df = (pa_federal_df.set_index(['fip', 'State', 'County'])
         .stack()
         .reset_index(name='PA')).rename({'level_3': "disasterNumber"}, axis = 1)

In [114]:
pa_federal_df = pa_federal_df[pa_federal_df["PA"] != 0]

In [115]:
pa_federal_df = pa_federal_df[pa_federal_df["disasterNumber"].astype(int).isin(disasterNumber_list)]

In [116]:
def correct_fips(col):
    if col < 10000:
        fip = "0" + str(col)
    else:
        fip = str(col)
    return fip

In [117]:
pa_federal_df["fip"] = pa_federal_df["fip"].apply(correct_fips)

In [118]:
pa_federal_df["County"] = pa_federal_df["County"].fillna("Statewide")

In [119]:
pa_federal_df = pa_federal_df.rename({'fip':'FIPS'}, axis = 1)

In [120]:
pa_federal_df_temp = pa_federal_df[["FIPS", "disasterNumber", "PA"]]

In [121]:
pa_federal_df_temp["disasterNumber"] = pa_federal_df_temp["disasterNumber"].astype(int)

<ipython-input-121-61c1690ee696>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pa_federal_df_temp["disasterNumber"] = pa_federal_df_temp["disasterNumber"].astype(int)


In [122]:
pa_federal_df_temp.head()

,FIPS,disasterNumber,PA
3,01000,1261,2.995779e+06
52,01000,1317,3.029702e+06
80,01000,1352,9.642729e+04
161,01000,1438,1.433254e+05
164,01000,1442,8.696048e+05


# Processing cdbg_df

In [123]:
cdbg_df = cdbg_df.loc[:, (cdbg_df != 0).any(axis=0)]

In [124]:
cdbg_df["fips"] = cdbg_df["fips"].apply(correct_fips)

In [125]:
cdbg_df = (cdbg_df.set_index(['fips', 'State', 'County'])
         .stack()
         .reset_index(name='CDBG')).rename({'level_3': "programFy"}, axis = 1)

In [126]:
cdbg_df = cdbg_df[cdbg_df["CDBG"] != 0]

In [127]:
cdbg_df["stateNumberCode"] = cdbg_df["fips"].str[:2].astype(int)
cdbg_df["countyCode"] = cdbg_df["fips"].str[-3:].astype(int)

In [128]:
cdbg_df_temp = cdbg_df[["stateNumberCode", "countyCode", "fips", 'programFy', 'CDBG']]

In [129]:
cdbg_df_temp = cdbg_df_temp.rename({'fips': 'FIPS'}, axis = 1)

In [130]:
cdbg_df_temp.head()

,stateNumberCode,countyCode,FIPS,programFy,CDBG
9,1,1,01001,2011,111064.084509
24,1,9,01009,2011,953950.536241
39,1,15,01015,2011,111064.084509
54,1,19,01019,2011,111064.084509
69,1,23,01023,2011,111064.084509


# Use data related to disasterNumber in disaster_df

In [131]:
disasterNumber_list = disaster_df["disasterNumber"].unique()

In [132]:
hma_df_merge = hma_df_merge[hma_df_merge["disasterNumber"].isin(disasterNumber_list)]
pa_federal_df_temp = pa_federal_df_temp[pa_federal_df_temp["disasterNumber"].isin(disasterNumber_list)]
final_ihp_df = final_ihp_df[final_ihp_df["disasterNumber"].isin(disasterNumber_list)]

In [133]:
final_ihp_df_temp = final_ihp_df[["stateNumberCode", "countyCode", "disasterNumber", "ihp", "ihp_cpi"]]

In [134]:
hma_df_merge["countyCode"] = hma_df_merge["countyCode"].astype(int)
hma_df_merge["disasterNumber"] = hma_df_merge["disasterNumber"].astype(int)

<ipython-input-134-f62d55c3adda>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hma_df_merge["countyCode"] = hma_df_merge["countyCode"].astype(int)
<ipython-input-134-f62d55c3adda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hma_df_merge["disasterNumber"] = hma_df_merge["disasterNumber"].astype(int)


# Merge Data

In [135]:
merge1 = pd.merge(hma_df_merge, final_ihp_df_temp, on = ["stateNumberCode", "countyCode", "disasterNumber"], how = 'outer')

In [136]:
merge1.head()

,stateNumberCode,state,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi
0,1,Alabama,0,848,1990.0,15794.0,0.0,NaN,NaN
1,1,Alabama,0,856,1990.0,22597.0,0.0,NaN,NaN
2,1,Alabama,0,861,1990.0,84583.0,0.0,NaN,NaN
3,1,Alabama,0,1013,1994.0,267669.0,0.0,NaN,NaN
4,1,Alabama,0,1019,1994.0,200738.0,0.0,NaN,NaN


In [137]:
pa_federal_df_temp.head()

,FIPS,disasterNumber,PA
3,01000,1261,2.995779e+06
52,01000,1317,3.029702e+06
80,01000,1352,9.642729e+04
161,01000,1438,1.433254e+05
164,01000,1442,8.696048e+05


In [138]:
pa_federal_df_temp["stateNumberCode"] = pa_federal_df_temp["FIPS"].str[:2].astype(int)
pa_federal_df_temp["countyCode"] = pa_federal_df_temp["FIPS"].str[-3:].astype(int)

In [139]:
pa_federal_df_temp.head()

,FIPS,disasterNumber,PA,stateNumberCode,countyCode
3,01000,1261,2.995779e+06,1,0
52,01000,1317,3.029702e+06,1,0
80,01000,1352,9.642729e+04,1,0
161,01000,1438,1.433254e+05,1,0
164,01000,1442,8.696048e+05,1,0


In [140]:
pa_federal_df_temp = pa_federal_df_temp[["stateNumberCode", "countyCode", "disasterNumber", "PA"]]

In [141]:
merge2 = pd.merge(merge1, pa_federal_df_temp, on = ["stateNumberCode", "countyCode", "disasterNumber"], how = 'outer')

In [142]:
merge2

,stateNumberCode,state,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA
0,1,Alabama,0,848,1990.0,15794.0,0.0,NaN,NaN,NaN
1,1,Alabama,0,856,1990.0,22597.0,0.0,NaN,NaN,NaN
2,1,Alabama,0,861,1990.0,84583.0,0.0,NaN,NaN,NaN
3,1,Alabama,0,1013,1994.0,267669.0,0.0,NaN,NaN,NaN
4,1,Alabama,0,1019,1994.0,200738.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
30924,56,NaN,1,4227,NaN,NaN,NaN,NaN,NaN,806590.433750
30925,56,NaN,35,4007,NaN,NaN,NaN,NaN,NaN,47790.712825
30926,56,NaN,19,4535,NaN,NaN,NaN,NaN,NaN,20186.610000
30927,56,NaN,45,4007,NaN,NaN,NaN,NaN,NaN,352935.817397


In [143]:
cdbg_df_temp = cdbg_df[["stateNumberCode", "countyCode", "programFy", "CDBG"]]

In [144]:
disaster_df = pd.merge(disaster_df, state_fips_temp, on = 'state', how = 'left')

In [145]:
disaster_df.head()

,disasterNumber,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,state,declarationDate,fyDeclared,disasterType,incidentType,title,incidentBeginDate,incidentEndDate,disasterCloseOutDate,declaredCountyArea,placeCode,hash,lastRefresh,id,stateNumberCode
0,3,0,1,1,1,LA,1953-05-29T00:00:00.000Z,1953,DR,Flood,FLOOD,1953-05-29T00:00:00.000Z,1953-05-29T00:00:00.000Z,1960-02-01T00:00:00.000Z,NaN,NaN,86f3e47785cb7acc51364d4535d36101,2020-09-23T19:21:12.990Z,5f6ba02931a8c6681cfc6942,22.0
1,1,0,1,1,1,GA,1953-05-02T00:00:00.000Z,1953,DR,Tornado,TORNADO,1953-05-02T00:00:00.000Z,1953-05-02T00:00:00.000Z,1954-06-01T00:00:00.000Z,NaN,NaN,1dcb40d0664d22d39de787b706b0fa69,2020-09-23T19:21:12.987Z,5f6ba02931a8c6681cfc6943,13.0
2,9,0,1,1,1,TX,1953-06-19T00:00:00.000Z,1953,DR,Flood,FLOOD,1953-06-19T00:00:00.000Z,1953-06-19T00:00:00.000Z,1958-01-01T00:00:00.000Z,NaN,NaN,5cc3af111f94e26100fca2605a9d0d26,2020-09-23T19:21:12.993Z,5f6ba02931a8c6681cfc6947,48.0
3,11,0,1,1,1,NH,1953-07-02T00:00:00.000Z,1953,DR,Fire,FOREST FIRE,1953-07-02T00:00:00.000Z,1953-07-02T00:00:00.000Z,1956-02-01T00:00:00.000Z,NaN,NaN,9d856a5b709986eb302e605ffbb10c7b,2020-09-23T19:21:12.994Z,5f6ba02931a8c6681cfc6949,33.0
4,4,0,1,1,1,MI,1953-06-02T00:00:00.000Z,1953,DR,Tornado,TORNADO,1953-06-02T00:00:00.000Z,1953-06-02T00:00:00.000Z,1956-02-01T00:00:00.000Z,NaN,NaN,1dbe5937a01fc74c8e699912e3f555cb,2020-09-23T19:21:12.990Z,5f6ba02931a8c6681cfc694f,26.0


In [146]:
state_fips_list = state_fips_temp["stateNumberCode"].unique()
disaster_df= disaster_df[disaster_df["stateNumberCode"].isin(state_fips_list)]

In [147]:
disaster_df_temp = disaster_df[["disasterNumber", "stateNumberCode", "fyDeclared"]]

In [148]:
disaster_df_temp = disaster_df_temp.rename({'fyDeclared': 'programFy'}, axis = 1)

In [149]:
disaster_df_temp = disaster_df_temp.drop_duplicates()

In [150]:
disaster_df_temp['stateNumberCode'] = disaster_df_temp['stateNumberCode'].astype(int)

In [151]:
merge2.head()

,stateNumberCode,state,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA
0,1,Alabama,0,848,1990.0,15794.0,0.0,NaN,NaN,NaN
1,1,Alabama,0,856,1990.0,22597.0,0.0,NaN,NaN,NaN
2,1,Alabama,0,861,1990.0,84583.0,0.0,NaN,NaN,NaN
3,1,Alabama,0,1013,1994.0,267669.0,0.0,NaN,NaN,NaN
4,1,Alabama,0,1019,1994.0,200738.0,0.0,NaN,NaN,NaN


In [152]:
merge3 = pd.merge(merge2, disaster_df_temp, on = ['disasterNumber', 'stateNumberCode'], how = 'left')

In [153]:
merge3.loc[merge3["programFy_x"].isna(), 'programFy_x'] = merge3['programFy_y']

In [154]:
programfy_done = merge3[merge3["programFy_x"].notna()] #merge 1
programfy_none = merge3[merge3["programFy_x"].isna()]

In [155]:
deal_ihp = programfy_none[programfy_none["ihp"].notna()] #merge 2
deal_pa = programfy_none[programfy_none["PA"].notna()] #merge 3

In [156]:
ihp_df_temp = ihp_df[["programFy", "disasterNumber", "ihp"]]

In [157]:
ihp_df_temp = ihp_df_temp.drop_duplicates()

In [158]:
programfy_done.head()

,stateNumberCode,state,countyCode,disasterNumber,programFy_x,hmgp,pdm,ihp,ihp_cpi,PA,programFy_y
0,1,Alabama,0,848,1990.0,15794.0,0.0,NaN,NaN,NaN,1990.0
1,1,Alabama,0,856,1990.0,22597.0,0.0,NaN,NaN,NaN,1990.0
2,1,Alabama,0,861,1990.0,84583.0,0.0,NaN,NaN,NaN,1990.0
3,1,Alabama,0,1013,1994.0,267669.0,0.0,NaN,NaN,NaN,1994.0
4,1,Alabama,0,1019,1994.0,200738.0,0.0,NaN,NaN,NaN,1994.0


In [159]:
programfy_done = programfy_done.rename({'programFy_x': 'programFy'}, axis = 1) #merge 1

In [160]:
cols = ["stateNumberCode", "countyCode", "disasterNumber", "programFy", "hmgp", "pdm", "ihp", "ihp_cpi", "PA"]

In [161]:
deal_ihp_ready = pd.merge(deal_ihp, ihp_df_temp, on = ["disasterNumber", "ihp"], how = 'left')

In [162]:
deal_ihp_ready.head()

,stateNumberCode,state,countyCode,disasterNumber,programFy_x,hmgp,pdm,ihp,ihp_cpi,PA,programFy_y,programFy
0,1,NaN,33,1551,NaN,NaN,NaN,7454.0,10087.614040,NaN,NaN,2004
1,1,NaN,47,1604,NaN,NaN,NaN,6358.0,8324.201732,NaN,NaN,2005
2,1,NaN,57,1561,NaN,NaN,NaN,1562.0,2113.878875,NaN,NaN,2004
3,1,NaN,59,1604,NaN,NaN,NaN,2358.0,3087.207877,NaN,NaN,2005
4,1,NaN,67,1604,NaN,NaN,NaN,2000.0,2618.496927,NaN,NaN,2005


In [163]:
deal_ihp_ready = deal_ihp_ready[cols] # Merge 2

In [164]:
# This deal_pa has disaster which happend in place not in the mainland.
deal_pa

,stateNumberCode,state,countyCode,disasterNumber,programFy_x,hmgp,pdm,ihp,ihp_cpi,PA,programFy_y
17967,60,NaN,50,1506,NaN,NaN,NaN,NaN,NaN,1.322926e+05,NaN
17968,60,NaN,50,4357,NaN,NaN,NaN,NaN,NaN,2.668699e+05,NaN
18849,64,NaN,60,1417,NaN,NaN,NaN,NaN,NaN,1.967027e+06,NaN
18850,64,NaN,60,1504,NaN,NaN,NaN,NaN,NaN,1.383508e+06,NaN
18851,64,NaN,60,1511,NaN,NaN,NaN,NaN,NaN,1.750327e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29974,78,NaN,20,4335,NaN,NaN,NaN,NaN,NaN,9.804145e+03,NaN
29975,78,NaN,20,4340,NaN,NaN,NaN,NaN,NaN,2.178903e+04,NaN
29976,78,NaN,30,1567,NaN,NaN,NaN,NaN,NaN,7.602610e+04,NaN
29977,78,NaN,30,4335,NaN,NaN,NaN,NaN,NaN,5.087165e+05,NaN


In [165]:
stateNumberCode_list = state_fips["stateNumberCode"].unique()

In [166]:
deal_pa[~deal_pa["stateNumberCode"].isin(stateNumberCode_list)]

,stateNumberCode,state,countyCode,disasterNumber,programFy_x,hmgp,pdm,ihp,ihp_cpi,PA,programFy_y
17967,60,NaN,50,1506,NaN,NaN,NaN,NaN,NaN,1.322926e+05,NaN
17968,60,NaN,50,4357,NaN,NaN,NaN,NaN,NaN,2.668699e+05,NaN
18849,64,NaN,60,1417,NaN,NaN,NaN,NaN,NaN,1.967027e+06,NaN
18850,64,NaN,60,1504,NaN,NaN,NaN,NaN,NaN,1.383508e+06,NaN
18851,64,NaN,60,1511,NaN,NaN,NaN,NaN,NaN,1.750327e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29974,78,NaN,20,4335,NaN,NaN,NaN,NaN,NaN,9.804145e+03,NaN
29975,78,NaN,20,4340,NaN,NaN,NaN,NaN,NaN,2.178903e+04,NaN
29976,78,NaN,30,1567,NaN,NaN,NaN,NaN,NaN,7.602610e+04,NaN
29977,78,NaN,30,4335,NaN,NaN,NaN,NaN,NaN,5.087165e+05,NaN


In [167]:
programfy_done.head()

,stateNumberCode,state,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA,programFy_y
0,1,Alabama,0,848,1990.0,15794.0,0.0,NaN,NaN,NaN,1990.0
1,1,Alabama,0,856,1990.0,22597.0,0.0,NaN,NaN,NaN,1990.0
2,1,Alabama,0,861,1990.0,84583.0,0.0,NaN,NaN,NaN,1990.0
3,1,Alabama,0,1013,1994.0,267669.0,0.0,NaN,NaN,NaN,1994.0
4,1,Alabama,0,1019,1994.0,200738.0,0.0,NaN,NaN,NaN,1994.0


In [168]:
deal_ihp_ready.head()

,stateNumberCode,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA
0,1,33,1551,2004,NaN,NaN,7454.0,10087.614040,NaN
1,1,47,1604,2005,NaN,NaN,6358.0,8324.201732,NaN
2,1,57,1561,2004,NaN,NaN,1562.0,2113.878875,NaN
3,1,59,1604,2005,NaN,NaN,2358.0,3087.207877,NaN
4,1,67,1604,2005,NaN,NaN,2000.0,2618.496927,NaN


In [169]:
programfy_done = programfy_done[["stateNumberCode", "countyCode", "disasterNumber", "programFy", "hmgp", "pdm", "ihp", "ihp_cpi","PA"]]

In [170]:
programfy_done

,stateNumberCode,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA
0,1,0,848,1990.0,15794.0,0.0,NaN,NaN,NaN
1,1,0,856,1990.0,22597.0,0.0,NaN,NaN,NaN
2,1,0,861,1990.0,84583.0,0.0,NaN,NaN,NaN
3,1,0,1013,1994.0,267669.0,0.0,NaN,NaN,NaN
4,1,0,1019,1994.0,200738.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
30924,56,1,4227,2015.0,NaN,NaN,NaN,NaN,806590.433750
30925,56,35,4007,2011.0,NaN,NaN,NaN,NaN,47790.712825
30926,56,19,4535,2020.0,NaN,NaN,NaN,NaN,20186.610000
30927,56,45,4007,2011.0,NaN,NaN,NaN,NaN,352935.817397


In [171]:
merge_hma_ihp_pa = pd.concat([programfy_done, deal_ihp_ready], ignore_index = True)

In [172]:
merge_hma_ihp_pa

,stateNumberCode,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA
0,1,0,848,1990.0,15794.0,0.0,NaN,NaN,NaN
1,1,0,856,1990.0,22597.0,0.0,NaN,NaN,NaN
2,1,0,861,1990.0,84583.0,0.0,NaN,NaN,NaN
3,1,0,1013,1994.0,267669.0,0.0,NaN,NaN,NaN
4,1,0,1019,1994.0,200738.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
30429,48,75,1607,2005.0,NaN,NaN,2000.0,2618.496927,NaN
30430,48,97,1607,2005.0,NaN,NaN,2000.0,2618.496927,NaN
30431,48,105,1603,2005.0,NaN,NaN,2000.0,2618.496927,NaN
30432,48,109,1607,2005.0,NaN,NaN,2000.0,2618.496927,NaN


In [173]:
cdbg_df_temp

,stateNumberCode,countyCode,programFy,CDBG
9,1,1,2011,1.110641e+05
24,1,9,2011,9.539505e+05
39,1,15,2011,1.110641e+05
54,1,19,2011,1.110641e+05
69,1,23,2011,1.110641e+05
...,...,...,...,...
15491,9,1,2013,1.364692e+08
15505,9,7,2012,8.934391e+06
15520,9,9,2012,6.140485e+07
15521,9,9,2013,1.686664e+08


In [174]:
merge_hma_ihp_pa["programFy"] = merge_hma_ihp_pa["programFy"].astype(int)

In [175]:
cdbg_df_temp["programFy"] = cdbg_df_temp["programFy"].astype(int)

<ipython-input-175-37eca0073f13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdbg_df_temp["programFy"] = cdbg_df_temp["programFy"].astype(int)


In [176]:
final_merge = pd.merge(merge_hma_ihp_pa, cdbg_df_temp, on = ['stateNumberCode', 'countyCode', 'programFy'], how = 'left')

In [177]:
final_merge["hmgp"] = final_merge["hmgp"].fillna(0)
final_merge["pdm"] = final_merge["pdm"].fillna(0)
final_merge["ihp"] = final_merge["ihp"].fillna(0)
final_merge["ihp_cpi"] = final_merge["ihp_cpi"].fillna(0)
final_merge["PA"] = final_merge["PA"].fillna(0)
final_merge["CDBG"] = final_merge["CDBG"].fillna(0)

In [178]:
final_merge

,stateNumberCode,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA,CDBG
0,1,0,848,1990,15794.0,0.0,0.0,0.000000,0.0,0.000000
1,1,0,856,1990,22597.0,0.0,0.0,0.000000,0.0,0.000000
2,1,0,861,1990,84583.0,0.0,0.0,0.000000,0.0,0.000000
3,1,0,1013,1994,267669.0,0.0,0.0,0.000000,0.0,0.000000
4,1,0,1019,1994,200738.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
30429,48,75,1607,2005,0.0,0.0,2000.0,2618.496927,0.0,0.000000
30430,48,97,1607,2005,0.0,0.0,2000.0,2618.496927,0.0,0.000000
30431,48,105,1603,2005,0.0,0.0,2000.0,2618.496927,0.0,243931.359422
30432,48,109,1607,2005,0.0,0.0,2000.0,2618.496927,0.0,0.000000


In [179]:
final_merge_stateName = pd.merge(final_merge, state_fips, on = 'stateNumberCode', how = 'left')

In [180]:
final_merge_stateName.head()

,stateNumberCode,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA,CDBG,state,state_name
0,1,0,848,1990,15794.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA
1,1,0,856,1990,22597.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA
2,1,0,861,1990,84583.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA
3,1,0,1013,1994,267669.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA
4,1,0,1019,1994,200738.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA


In [181]:
final_merge_df = pd.merge(final_merge_stateName, county_fips, on = ["stateNumberCode", "countyCode"], how = "left")

In [182]:
final_merge_df.head()

,stateNumberCode,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA,CDBG,state,state_name,county
0,1,0,848,1990,15794.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama
1,1,0,856,1990,22597.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama
2,1,0,861,1990,84583.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama
3,1,0,1013,1994,267669.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama
4,1,0,1019,1994,200738.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama


# Adjusting Consumer Price Index(CPI)
This is from https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-

In [183]:
cpi_df = pd.read_excel(file_path + 'ave_cpi_by_year.xlsx')
cpi_df["year"] = cpi_df["year"].astype(int)
cpi_df = cpi_df.rename({'year': 'programFy'}, axis = 1)

In [184]:
final_merge_df_add_cpi = pd.merge(final_merge_df, cpi_df, on = 'programFy', how = 'left')

In [185]:
final_merge_df_add_cpi.head()

,stateNumberCode,countyCode,disasterNumber,programFy,hmgp,pdm,ihp,ihp_cpi,PA,CDBG,state,state_name,county,ave_cpi
0,1,0,848,1990,15794.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama,130.7
1,1,0,856,1990,22597.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama,130.7
2,1,0,861,1990,84583.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama,130.7
3,1,0,1013,1994,267669.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama,148.2
4,1,0,1019,1994,200738.0,0.0,0.0,0.0,0.0,0.0,AL,ALABAMA,Alabama,148.2


In [186]:
cpi_df["cpi_2019_base"] =  255.7 / cpi_df["ave_cpi"]

In [187]:
final_df = pd.merge(final_merge_df, cpi_df, on = 'programFy', how = 'left')

In [188]:
# CPI adjust to 2019
final_df["hmgp_cpi_2019"] = final_df["cpi_2019_base"] * final_df["hmgp"]
final_df["ihp_cpi_2019"] = final_df["cpi_2019_base"] * final_df["ihp"]
final_df["pdm_cpi_2019"] = final_df["cpi_2019_base"] * final_df["pdm"]
final_df["CDBG_cpi_2019"] = final_df["cpi_2019_base"] * final_df["CDBG"]
final_df["PA_cpi_2019"] = final_df["cpi_2019_base"] * final_df["PA"]

In [189]:
final_df = final_df[["stateNumberCode", "state", "state_name", "countyCode", "county", "disasterNumber", "programFy", "hmgp", "pdm", "ihp", "PA", "CDBG", "ave_cpi", "cpi_2019_base", "hmgp_cpi_2019", "ihp_cpi_2019", "pdm_cpi_2019", "PA_cpi_2019", "CDBG_cpi_2019"]]

In [193]:
final_df.head()

,stateNumberCode,state,state_name,countyCode,county,disasterNumber,programFy,hmgp,pdm,ihp,PA,CDBG,ave_cpi,cpi_2019_base,hmgp_cpi_2019,ihp_cpi_2019,pdm_cpi_2019,PA_cpi_2019,CDBG_cpi_2019
0,1,AL,ALABAMA,0,Alabama,848,1990,15794.0,0.0,0.0,0.0,0.0,130.7,1.956389,30899.202754,0.0,0.0,0.0,0.0
1,1,AL,ALABAMA,0,Alabama,856,1990,22597.0,0.0,0.0,0.0,0.0,130.7,1.956389,44208.514920,0.0,0.0,0.0,0.0
2,1,AL,ALABAMA,0,Alabama,861,1990,84583.0,0.0,0.0,0.0,0.0,130.7,1.956389,165477.223412,0.0,0.0,0.0,0.0
3,1,AL,ALABAMA,0,Alabama,1013,1994,267669.0,0.0,0.0,0.0,0.0,148.2,1.725371,461828.362348,0.0,0.0,0.0,0.0
4,1,AL,ALABAMA,0,Alabama,1019,1994,200738.0,0.0,0.0,0.0,0.0,148.2,1.725371,346347.547908,0.0,0.0,0.0,0.0


Now everything is good to go! Let's save this data!

In [ ]:
#final_df.to_csv(file_path + 'merged_df.csv')